In [41]:
"""
Solves the incompressible Navier-Stokes equations in conjunction with
an advection equation in a closed box.

Momentum:           ∂u/∂t + (u ⋅ ∇) u = − 1/ρ ∇p + ν ∇²u + f

Incompressibility:  ∇ ⋅ u = 0

Advection:          ∂s/∂t + (u ⋅ ∇) s = α ∇²s + i

u:  Velocity (2d vector)
p:  Pressure
f:  Forcing (here due to Buoyancy)
ν:  Kinematic Viscosity
ρ:  Density (here =1.0)
t:  Time
∇:  Nabla operator (defining nonlinear convection, gradient and divergence)
∇²: Laplace Operator
s:  Concentration of a species (here hot smoke)
α:  Diffusivity of the embedded species
i:  Inflow of hot smoke into the domain.

----------

Scenario


    +----------------------------------------------+
    |                                              |
    |                                              |
    |                                              |
    |                                              |
    |                                              |
    |                                              |
    |                                              |
    |                                              |
    |                                              |
    |                                              |
    |                                              |
    |                                              |
    |           _                                  |
    |          / \                                 |
    |         |   |                                |
    |          \_/                                 |
    |                                              |
    +----------------------------------------------+

-> Domain is square and closed-off (wall BC everywhere)
-> Initially, the fluid is at rest
-> Initially, the concentration of smoke is zero everywhere
-> There is a continuous inflow of hot smoke in a small circular
   patch in the bottom left of the domain
-> The hot smoke exerts a force on the fluid due to Buyancy
-> This makes the fluid flow upwards and create a plume pattern

-------

Solution strategy:

Initialize the fluid velocity vectors to zero on a Staggered Grid.

Initialize the smoke density to zero on a Centered Grid.

1. Advect the smoke density by a MacCormack step

2. Add the inflow of hot smoke to the smoke density field

3. Compute the Buoyancy force by re-sampling the centered
   smoke densities on the staggered velocities

4. Convect the fluid by means of a semi-lagrangian self-avection
   step 

5. Add the Buoyancy force to the fluid

6. Make the fluid incompressible

7. Repeat


Note, that we did not apply any diffusion on both the fluid and
the smoke concentration. This is done for simplicity, the involved
convection/advection procedures introduce considerable numerical
diffusion which stabilize the simulation.
"""

from phi.torch import flow
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
%matplotlib qt

N_TIME_STEPS = 100

def main():
    velocity = flow.StaggeredGrid(
        values=(0.0, 0.0),
        extrapolation=0.0,
        x=64,
        y=64,
        bounds=flow.Box(x=100, y=100),
    )
    smoke = flow.CenteredGrid(
        values=0.0,
        extrapolation=flow.extrapolation.BOUNDARY,
        x=200,
        y=200,
        bounds=flow.Box(x=100, y=100),
    )
    inflow = 0.2 * flow.CenteredGrid(
        values=flow.SoftGeometryMask(
            flow.Sphere(
                x=40,
                y=9.5,
                radius=5,
            )
        ),
        extrapolation=0.0,
        bounds=smoke.bounds,
        resolution=smoke.resolution,
    )

    @flow.math.jit_compile
    def step(velocity_prev, smoke_prev, dt=1.0):
        smoke_next = flow.advect.mac_cormack(smoke_prev, velocity_prev, dt) + inflow
        buoyancy_force = smoke_next * (0.0, 0.1) @ velocity
        velocity_tent = flow.advect.semi_lagrangian(velocity_prev, velocity_prev, dt) + buoyancy_force * dt
        velocity_next, pressure = flow.fluid.make_incompressible(velocity_tent)
        return velocity_next, smoke_next
    
    plt.style.use("dark_background")
    
    for _ in tqdm(range(N_TIME_STEPS)):
        velocity, smoke = step(velocity, smoke)
        smoke_values_extracted = smoke.values.numpy("y,x")
        plt.imshow(smoke_values_extracted, origin="lower")
        
        # Plot velocity vectors
        x, y = np.meshgrid(np.arange(5, 100, 10), np.arange(5, 100, 10))  # Adjusted to avoid boundary values
        u = velocity.staggered_tensor().data[{"xˢ": slice(5, None), "yˢ": slice(5, None), "vectorᶜ": 0}]
        v = velocity.staggered_tensor().data[{"xˢ": slice(5, None), "yˢ": slice(5, None), "vectorᶜ": 1}]
        plt.quiver(x, y, u.values.numpy(), v.values.numpy(), scale=20, color='red')  # Use .values.numpy() to get the numpy array
        
        plt.draw()
        plt.pause(0.01)
        plt.clf()  # Clear the plot for the next frame



if __name__ == "__main__":
    main()

<>:1: SyntaxWarning: invalid escape sequence '\ '
<>:1: SyntaxWarning: invalid escape sequence '\ '
C:\Users\jortd\AppData\Local\Temp\ipykernel_5452\2197730056.py:1: SyntaxWarning: invalid escape sequence '\ '
  """
C:\Users\jortd\AppData\Local\Temp\ipykernel_5452\2197730056.py:111: DeprecationWarning: HardGeometryMask and SoftGeometryMask are deprecated. Use field.mask or field.resample instead.
  values=flow.SoftGeometryMask(
  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\jortd\PycharmProjects\LowCostPIV\venv\Lib\site-packages\phi\field\_field_math.py:521: FutureWarning: Instance checks on Grid are deprecated and will be removed in version 3.0. Use the methods instance.is_grid, instance.is_point_cloud, instance.is_centered and instance.is_staggered instead.
  if isinstance(grid, Grid):
C:\Users\jortd\PycharmProjects\LowCostPIV\venv\Lib\site-packages\phi\field\_grid.py:476: FutureWarning: Instance checks on StaggeredGrid are deprecated and will be removed in version 3.0. Use the meth

TypeError: Method TensorStack.TensorStack.values.numpy.name() does not exist.

In [42]:
import numpy as np
import matplotlib.pyplot as plt
from phi.flow import *
from tqdm import tqdm

N_TIME_STEPS = 100

def main():
    velocity = StaggeredGrid(
        values=(0.0, 0.0),
        extrapolation=extrapolation.BOUNDARY,
        box=Box(x=100, y=100),
        resolution=64
    )
    smoke = CenteredGrid(
        values=0.0,
        extrapolation=extrapolation.BOUNDARY,
        box=Box(x=100, y=100),
        resolution=64
    )
    inflow = 0.2 * CenteredGrid(
        values=SoftGeometryMask(
            Sphere(
                center=(40, 9.5),
                radius=5,
            )
        ),
        extrapolation=extrapolation.ZERO,
        box=Box(x=100, y=100),
        resolution=64,
    )

    @jit_compile
    def step(velocity_prev, smoke_prev, dt=1.0):
        smoke_next = advect.mac_cormack(smoke_prev, velocity_prev, dt) + inflow
        buoyancy_force = smoke_next * (0.0, 0.1) @ velocity
        velocity_tent = advect.semi_lagrangian(velocity_prev, velocity_prev, dt) + buoyancy_force * dt
        velocity_next, _ = fluid.make_incompressible(velocity_tent)
        return velocity_next, smoke_next
    
    plt.style.use("dark_background")
    
    for _ in tqdm(range(N_TIME_STEPS)):
        velocity, smoke = step(velocity, smoke)
        smoke_values_extracted = smoke.values.numpy("y,x")
        plt.imshow(smoke_values_extracted, origin="lower")
        
        # Plot velocity vectors
        x, y = np.meshgrid(np.arange(5, 100, 10), np.arange(5, 100, 10))  # Adjusted to avoid boundary values
        u = velocity.staggered_tensor().data[{"xˢ": slice(5, None), "yˢ": slice(5, None), "vectorᶜ": 0}]
        v = velocity.staggered_tensor().data[{"xˢ": slice(5, None), "yˢ": slice(5, None), "vectorᶜ": 1}]
        plt.quiver(x, y, u.values.numpy(), v.values.numpy(), scale=20, color='red')  # Use .values.numpy() to get the numpy array
        
        plt.draw()
        plt.pause(0.01)
        plt.clf()  # Clear the plot for the next frame

if __name__ == "__main__":
    main()


AssertionError: Cannot specify keyword resolution and integer resolution at the same time.

In [40]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from phi.flow import *

# Define simulation parameters
resolution = 64
domain = domain(x=resolution, y=resolution)
velocity = CenteredGrid(Noise(), domain.bounds)
density = CenteredGrid(0, domain.bounds)

# Define the smoke simulation
smoke = (
    dict(density=density, velocity=velocity),
    advect.semi_lagrangian(density, velocity, dt=0.1),
    advect.semi_lagrangian(velocity, velocity, dt=0.1) + 0.01 * random_vector())
world = World(smoke)

# Define a function to update the plot
def update_plot(frame):
    world.step()
    ax.clear()
    ax.quiver(*velocity.with_data(velocity.values).points.data, color='r')
    ax.set_title(f'Frame {frame+1}')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)

# Create the plot
fig, ax = plt.subplots()
ani = animation.FuncAnimation(fig, update_plot, frames=100, interval=50)
plt.show()

NameError: name 'domain' is not defined